In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.modeling.models import Sersic2D
from astropy.convolution import convolve
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/nasekhian/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/xm/sl735y1n4m30kfvcxl7h4p6w0000gn/T/ipykernel_64376/1673434397.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Set a fixed seed for reproducibility
np.random.seed(42)

# Load the coordinates from the CSV file
coordinates = pd.read_csv('coordinates.csv')

# Define the number of galaxies to simulate
num_galaxies = 100

# Define lists to store input parameters
sersic_index_list = []
effective_radius_list = []
amplitude_list = []
ellipticity_list = []
theta_list = []
total_flux_list = []
magnitude_list = []


In [6]:

# Loop over each row in the DataFrame
for index, row in coordinates.iterrows():
    # Define the properties of the simulated galaxy for this iteration
    sersic_index = np.random.uniform(1, 4)  # Random Sersic index between 1 and 4
    effective_radius = np.random.uniform(2, 4)  # Random effective radius between 1 and 10
    amplitude = np.random.uniform(0.001, 0.01)  # Random amplitude between 0.1 and 1
    ellip = np.random.uniform(0.2, 0.9)  # Random ellipticity between 0 and 0.5
    theta = np.radians(np.random.uniform(0, 180))  # Random angle between 0 and 180 degrees

    # Specify the input file names
    galaxy_file = 'sci_f444w_22450.fits'
    psf_file = 'ePSFf444w.fits'

    # Load data
    galaxy_data, galaxy_header = fits.getdata(galaxy_file, header=True)
    psf_data = fits.getdata(psf_file)
    y, x = np.mgrid[:galaxy_data.shape[0], :galaxy_data.shape[1]]

    # Create a Sersic model for the simulated galaxy
    galaxy_model = Sersic2D(amplitude=amplitude, r_eff=effective_radius, n=sersic_index,
                             x_0=row['x0'], y_0=row['y0'], ellip=ellip, theta=theta)
    simulated_galaxy = galaxy_model(x, y)
    convolved_simulated = convolve(simulated_galaxy, psf_data, boundary='extend')

    # Calculate total flux
    total_flux = np.sum(simulated_galaxy)

    # Calculate magnitude
    magnitude = -2.5 * np.log10(total_flux) + 23.9

    # Append the input parameters to the lists
    sersic_index_list.append(sersic_index)
    effective_radius_list.append(effective_radius)
    amplitude_list.append(amplitude)
    ellipticity_list.append(ellip)
    theta_list.append(theta)
    total_flux_list.append(total_flux)
    magnitude_list.append(magnitude)

    # Save the convolved simulated galaxy fits file in the relevant folder
    simulation_dir = f'simulation_{index}'
    if not os.path.exists(simulation_dir):
        os.makedirs(simulation_dir)
    convolved_output_path = os.path.join(simulation_dir, f'simulated_galaxy_{index}.fits')
    fits.writeto(convolved_output_path, convolved_simulated, header=galaxy_header, overwrite=True)

    # Save the original simulated galaxy fits file in the same folder
    simulated_output_path = os.path.join(simulation_dir, f'original_simulated_galaxy_{index}.fits')
    fits.writeto(simulated_output_path, simulated_galaxy, header=galaxy_header, overwrite=True)

    # Add the simulated convolved galaxy image to the original science image
    final_image = galaxy_data + convolved_simulated

    # Save the final image with the simulated galaxy in the relevant folder
    final_output_path = os.path.join(simulation_dir, f'sci_withsimulated_{index}.fits')
    fits.writeto(final_output_path, final_image, header=galaxy_header, overwrite=True)

# Create a DataFrame from the lists of input parameters
input_params_df = pd.DataFrame({
    'Sersic_Index': sersic_index_list,
    'Effective_Radius': effective_radius_list,
    'Amplitude': amplitude_list,
    'Ellipticity': ellipticity_list,
    'Theta': theta_list,
    'Total_Flux': total_flux_list,
    'Magnitude': magnitude_list
})

# Set the index of the DataFrame to be the simulation number
input_params_df.index.name = 'Simulation_Number'

# Display the DataFrame
print(input_params_df)


                   Sersic_Index  Effective_Radius  Amplitude  Ellipticity  \
Simulation_Number                                                           
0                      2.060057          3.167312   0.001700     0.882076   
1                      2.060057          3.167312   0.001700     0.882076   
2                      3.094485          3.072193   0.003786     0.769657   
3                      1.487851          3.821854   0.008403     0.864860   
4                      2.840246          2.836486   0.009395     0.806245   
...                         ...               ...        ...          ...   
97                     1.786792          3.190156   0.001463     0.547456   
98                     2.002732          3.541824   0.001959     0.252596   
99                     2.486474          3.376805   0.004913     0.372481   
100                    3.398248          3.389393   0.003449     0.613161   
101                    1.274746          3.834627   0.002231     0.865166   

In [7]:
pd.concat([coordinates,input_params_df],axis=1)

,Unnamed: 0,x0,y0,Sersic_Index,Effective_Radius,Amplitude,Ellipticity,Theta,Total_Flux,Magnitude
0,1.0,443.0,526.0,2.060057,3.167312,0.001700,0.882076,3.098272,0.113381,26.263645
1,2.0,403.0,121.0,2.060057,3.167312,0.001700,0.882076,3.098272,0.113381,26.263645
2,3.0,141.0,415.0,3.094485,3.072193,0.003786,0.769657,2.151146,1.447850,23.498191
3,4.0,417.0,404.0,1.487851,3.821854,0.008403,0.864860,2.279915,0.319759,25.137945
4,5.0,219.0,143.0,2.840246,2.836486,0.009395,0.806245,0.142059,2.211699,23.038185
...,...,...,...,...,...,...,...,...,...,...
97,98.0,461.0,519.0,1.786792,3.190156,0.001463,0.547456,1.875037,0.129184,26.121977
98,99.0,279.0,419.0,2.002732,3.541824,0.001959,0.252596,2.287672,0.351669,25.034665
99,100.0,182.0,500.0,2.486474,3.376805,0.004913,0.372481,2.573286,1.067101,23.829486
100,NaN,NaN,NaN,3.398248,3.389393,0.003449,0.613161,1.134033,2.476475,22.915415


In [8]:

def crop_data_and_save(input_filename, center_x, center_y, simulation_dir):
    # Load the original FITS data
    data, header = fits.getdata(input_filename, header=True)

    # Define the size of the cropped region
    crop_size = 200  # Assuming you want a 200x200 pixels crop

    # Calculate the starting and ending indices along each axis
    start_x = center_x - crop_size // 2
    end_x = start_x + crop_size
    start_y = center_y - crop_size // 2
    end_y = start_y + crop_size

    # Crop the data
    cropped_data = data[start_y:end_y, start_x:end_x]

    # Generate the output filename
    output_filename = os.path.join(simulation_dir, f'cropped_sci_with_galaxy_{os.path.basename(input_filename).split("_")[-1]}')

    # Write the cropped data to a new FITS file
    fits.writeto(output_filename, cropped_data, header, overwrite=True)

# Process each simulation
for index, row in coordinates.iterrows():
    # Define the central coordinates for this simulation
    center_x = row['x0']
    center_y = row['y0']

    # Define the directory for this simulation
    simulation_dir = f'simulation_{index}'

    # Define the input filename
    input_filename = os.path.join(simulation_dir, f'sci_withsimulated_{index}.fits')

    # Crop and save the data
    crop_data_and_save(input_filename, center_x, center_y, simulation_dir)
